# Use Search with Gemini API

This notebook demonstrates how to integrate Google Search capabilities with Gemini API. We will explore various search scenarios and learn how to effectively combine AI reasoning with real-time web information.

We are not limited to Gemini’s pre-trained knowledge—using search tools allows Gemini to pull in current events, check for the latest data, and verify or expand on information it would not otherwise have. This is particularly useful for time-sensitive queries, like looking up sports scores, stock prices, or breaking news.

Rather than manually sending search requests and parsing them, the goal is to let Gemini handle reasoning and tool selection automatically - Google Search becomes just one more function Gemini can call when needed.

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool

# Load environment variables
load_dotenv()

# Set up API key
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

GenAITool acts as a formal declaration that tells Gemini what external services it is allowed to use.

### Initialize the Gemini model
Before running any queries, we need to create an instance of the Gemini model.

In [2]:
# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",  # Specify the exact Gemini model version
    temperature=0.7  # Controls randomness vs. determinism of model outputs
)

print("✅ Gemini model initialized successfully")

✅ Gemini model initialized successfully


`ChatGoogleGenerativeAI` is the main LangChain object responsible for handling prompts, managing tool calls, and receiving responses from Gemini.
- We choose a specific Gemini variant. Flash models prioritize speed over size or depth, which is often preferred in conversational agents where latency matters.


### Configure the search tool
Before sending prompts to Gemini, it’s important to explicitly define which external tools the model is allowed to use. Gemini itself doesn’t automatically “know” that Google Search is allowed—it needs to be configured through a tool declaration.

In [3]:
# Configure the search tool
search_tool = GenAITool(google_search={})

print("🔍 Google Search tool configured")
print(f"Tool type: {type(search_tool)}")  # Check the tool’s Python object type
print(f"Tool configuration: {search_tool}")  # Display the internal configuration state


🔍 Google Search tool configured
Tool type: <class 'google.ai.generativelanguage_v1beta.types.content.Tool'>
Tool configuration: google_search {
}



GenAITool is a configuration object that defines what tools the model can use.
- `google_search={}`: Enables Google Search functionality. Leaving it empty is intentional. It keeps things flexible and avoids over-constraining the search behavior. If we later want Gemini to prioritize results from a specific country or use a limited result count, we would add key-value pairs here. The empty dictionary uses default search parameters


### Basic search example
Let's start with a basic example asking about current technology trends.
This demonstrates how Gemini can search for and synthesize recent information.


In [4]:
# Example: Technology trends
response = llm.invoke(
    "What are the latest developments in artificial intelligence in 2025?",
    tools=[search_tool]  # Provide Gemini with access to Google Search capability for this query
)

print("🤖 Query: Latest AI developments in 2025")
print("📝 Response:")
print(response.content)
print("\n" + "="*50 + "\n")

🤖 Query: Latest AI developments in 2025
📝 Response:
Based on the search results, here are some of the latest developments in artificial intelligence as of July 2025:

**1. AI Models Becoming More Capable and Useful:**

*   AI models are faster and more efficient, with large-scale "frontier models" handling tasks from writing to coding, and specialized models tailored for specific industries.
*   Models with advanced reasoning capabilities can solve complex problems with logical steps, useful in science, coding, math, law, and medicine.
*   OpenAI has revealed that GPT-5 will unify advancements from several specialized models into a single, more capable system, improving versatility across tasks. GPT-5 is expected to launch later in 2025.

**2. AI-Powered Agents and Companions:**

*   A new generation of AI-powered agents can handle tasks on your behalf, transforming business processes and revolutionizing work management.
*   AI companions, like Microsoft Copilot, support you throughout

* `llm.invoke()` sends both the query and tool access info to Gemini.
  * `tools=[search_tool]`: Provides Google Search capability

Gemini automatically determines if search is needed, performs web searches, synthesizes information from multiple sources and provides a comprehensive answer.

### Real-time financial information
Financial markets change rapidly, making this a perfect use case for search-enabled AI. Gemini allows for retrieving the current performance of specific stocks.

In [5]:
# Example: Financial markets
response = llm.invoke(
    "What is the current performance of major tech stocks like Apple, Microsoft, and Google today?",
    tools=[search_tool]
)

print("📈 Query: Current tech stock performance")
print("📝 Response:")
print(response.content)
print("\n" + "="*50 + "\n")

📈 Query: Current tech stock performance
📝 Response:
Here's a summary of the current performance of Apple, Microsoft, and Google stocks as of July 16, 2025:

**Apple (AAPL)**

*   **Current Price:** \$210.16
*   **Change Today:** Increased by 0.47%
*   **Day's High/Low:** \$212.4 / \$208.64

**Microsoft (MSFT)**

*   **Current Price:** \$505.62
*   **Change Today:** Decreased by -0.04%

**Google (GOOG)**

*   **Current Price:** \$183.77
*   **Change Today:** Increased by 0.37%




The process demonstrates context awareness - understanding that “performance” in this query means looking for numbers like price changes, and market cap fluctuations, triggering the search functionality without hardcoding it.

Gemini is responsible for both querying the web and distilling complex financial data into an understandable, conversational response. We do not need to write custom API calls to financial market services; the search tool handles this through generalized search behavior. Also, instead of just dumping raw search results, Gemini crafts a human-readable answer based on financial context.




### Breaking news and current events
Search-enabled Gemini excels at providing current news summaries and analysis.

In [6]:
# Example: Current events
response = llm.invoke(
    "What are the most significant scientific breakthroughs announced in the past month?",
    tools=[search_tool]
)

print("🔬 Query: Recent scientific breakthroughs")
print("📝 Response:")
print(response.content)
print("\n" + "="*50 + "\n")

🔬 Query: Recent scientific breakthroughs
📝 Response:
Here are some of the most significant scientific breakthroughs announced in the past month:

*   **Black Hole Merger:** Scientists have detected the most massive black hole merger ever observed, resulting in a new black hole 225 times the mass of the sun. This discovery could provide evidence of a rare type of black hole.
*   **Ancient Glacier Ice:** The oldest known ice in Western Europe, dating back over 12,000 years to the last Ice Age, has been revealed by an ancient glacier high in the French Alps.
*   **AI Designs Custom Proteins:** Artificial intelligence can now design custom proteins in seconds, a process that previously took years, potentially accelerating cures for diseases like cancer and antibiotic-resistant infections.
*   **Parkinson's Detection by Dogs:** Dogs trained to detect Parkinson's disease using scent have demonstrated remarkable accuracy in identifying skin swabs from people with the disease.
*   **Exoplanet 

The prompt’s use of “past month” signals Gemini to prioritize recent search results over older ones. Google Search’s underlying system already ranks relevant results; Gemini layers summarization on top of this to avoid simply echoing headlines.

Recognizing what counts as a “scientific breakthrough” involves understanding scientific domains like medicine, physics, or AI research. Gemini’s language model handles that classification.


### Comparing multiple options
Search-enabled Gemini can gather information about different options and provide comparisons.


In [7]:
# Example: Product comparison
response = llm.invoke(
    "Compare the latest smartphone releases from Samsung, Apple, and Google in terms of features, price, and reviews",
    tools=[search_tool]
)

print("📱 Query: Latest smartphone comparison")
print("📝 Response:")
print(response.content)
print("\n" + "="*50 + "\n")

📱 Query: Latest smartphone comparison
📝 Response:
['Okay, let\'s compare the latest smartphone releases from Samsung, Apple, and Google, focusing on features, price, and reviews. Since the definition of "latest" can shift rapidly, I\'ll aim to provide information based on releases up to my current knowledge cutoff, and I\'ll use searches to get the most up-to-date information possible.', 'Okay, here\'s a comparison of the latest smartphone releases from Samsung, Apple, and Google, based on the search results. Note that "latest" can be subjective, so I\'m focusing on releases and information from the past few months (mid-2025).\n\n**Samsung**\n\n*   **Latest Releases:** Samsung has been busy with several releases, including the Galaxy S25 series (S25, S25+, S25 Ultra, and even an S25 Edge), the Galaxy Z Flip 6 and Z Fold 6, and more budget-friendly options like the Galaxy A series (e.g., A36, A26, A56). There are also FE (Fan Edition) versions sometimes.\n*   **Features:**\n    *   **Ga

The query is not just asking about one product or feature but comparing across brands and specs. Gemini identifies what comparison categories make sense - typically features, price, and reviews—based on the query.


### Location-specific queries
Search capabilities are particularly useful for location-based information that changes frequently.

In [8]:
# Example: Travel information
response = llm.invoke(
    "What are the current travel restrictions and requirements for visiting Japan from the United States?",
    tools=[search_tool]
)

print("✈️ Query: Current travel restrictions for Japan")
print("📝 Response:")
print(response.content)
print("\n" + "="*50 + "\n")

✈️ Query: Current travel restrictions for Japan
📝 Response:
As of July 17, 2025, the following travel restrictions and requirements apply for visiting Japan from the United States:

*   **COVID-19 related restrictions:** There are no COVID-19 testing, proof of vaccination, or quarantine requirements to travel to Japan.
*   **Visa:** U.S. citizens do not need a visa for short-term tourist visits (up to three months).
*   **Passport:** Your passport must be valid for the entire duration of your stay in Japan. You need one blank passport page for the entry stamp.
*   **Return ticket:** Even if a visa is not required, you should have proof of a return or onward ticket.
*   **Customs declaration:** All foreign visitors entering Japan must make declarations for immigration and customs clearance. This can be done in advance using the Visit Japan Web online system.
*   **Currency restrictions:** If you are entering or exiting Japan with ¥1,000,000 or more, you need to declare it.
*   **Medicat

The prompt specifies both the destination (Japan) and the origin (United States), guiding Gemini to search for route-specific regulations.

### Analytical market insights
In some scenarios, users want not just facts, but AI-powered insights and forecasts. This example focuses on the renewable energy sector, where up-to-date market reports and future trend predictions are both key.

In [9]:
# Example: Complex analytical query
response = llm.invoke(
    "Analyze the current global renewable energy market trends and predict future growth areas",
    tools=[search_tool]
)

print("🌱 Query: Renewable energy market analysis")
print("📝 Response:")
print(response.content)
print("\n" + "="*50 + "\n")

🌱 Query: Renewable energy market analysis
📝 Response:
['Okay, I can analyze the current global renewable energy market trends and predict future growth areas. To give you the most up-to-date and comprehensive analysis, I will use search queries to gather information on the latest market reports, trends, and expert forecasts.', "Here's an analysis of the current global renewable energy market trends and a prediction of future growth areas:\n\n**Current Global Renewable Energy Market Trends:**\n\n*   **Dominance of Solar and Wind:** Solar and wind energy continue to lead in new capacity additions. Solar PV capacity is expected to continue its strong growth, potentially surpassing hydropower and nuclear.\n*   **Record Investments:** Global investment in renewables reached record levels in 2023, driven by policy support and increasing demand for clean energy. Investments are flowing into manufacturing, particularly for solar PV and batteries.\n*   **Falling Costs:** Technological advanceme

The model is tasked with both describing current conditions and making future predictions based on real-world data. While Gemini doesn’t run hard predictive models, it uses reasoning based on fresh data to suggest likely growth areas.


### Handling potential issues
When building real-world applications using Gemini with live search functionality, it’s important to plan for things not always working perfectly. Internet-connected tools are prone to occasional network hiccups, API key issues, rate limits, or backend outages.

In [10]:
def safe_search_query(query, max_retries=3):
    """
    Safely execute a search query using Gemini with Google Search, including error handling and retry attempts.
    """
    for attempt in range(max_retries):
        try:
            # Attempt to send the query to Gemini with search capabilities
            response = llm.invoke(query, tools=[search_tool])
            return response  # Return successful response immediately
        except Exception as e:
            # Handle and log the error during this attempt
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            # If this was the last allowed attempt, give up gracefully
            if attempt == max_retries - 1:
                print("Max retries reached. Please check your API key and connection.")
                return None  # No response available after retries
    return None

# Example usage with a real query about space exploration
response = safe_search_query("What are the latest updates in space exploration missions?")
if response:
    print("🚀 Query: Latest space exploration updates")
    print("📝 Response:")
    print(response.content)
else:
    print("❌ Query failed after retries")

🚀 Query: Latest space exploration updates
📝 Response:
Here are some of the latest updates in space exploration missions as of July 17, 2025:

**Missions in Progress:**

*   **Mars Exploration:**
    *   NASA's Perseverance rover is actively searching for past life and collecting samples for future return to Earth.
    *   China's Tianwen-1 continues to orbit and study the planet.
    *   The United Arab Emirates' Hope orbiter is studying the Martian atmosphere.
    *   NASA's Curiosity rover explores an ancient lake bed.
*   **Asteroid Exploration:**
    *   NASA's Lucy mission is exploring asteroids that share Jupiter's orbit.
    *   Japan's Hayabusa2 is en route to two more asteroids after returning a sample from asteroid Ryugu in 2020.
*   **Solar Missions:**
    *   India's Aditya-L1 is studying the Sun.
    *   NASA's Parker Solar Probe is studying the Sun's atmosphere.
*   **Kuiper Belt:**
    *   NASA's New Horizons is exploring the Kuiper Belt.

**Upcoming Missions:**

*   **L

If all retry attempts fail, the function reports that and exits cleanly with `None` instead of crashing the application.

### Optimizing search queries
Simply throwing a vague question at Gemini won’t always yield the best results. Crafting clear, specific, and time-bounded queries helps in improving the relevance and precision of Gemini’s answers and reducing unnecessary web traffic or search load.




In [11]:
# Example of optimized query structure
optimized_queries = [
    "What are the top 3 cybersecurity threats identified in enterprise reports from the last 6 months?",
    "Compare the environmental impact of electric vs hydrogen vehicles based on 2024 studies",
    "What are the latest FDA approvals for cancer treatments announced this year?"
]

# Loop through each query
for query in optimized_queries:
    print(f"🎯 Optimized Query: {query}")
    response = llm.invoke(query, tools=[search_tool])  # Execute the query with search tool enabled
    print(f"📝 Response Preview: {response.content[:200]}...")
    print("\n" + "-"*30 + "\n")

🎯 Optimized Query: What are the top 3 cybersecurity threats identified in enterprise reports from the last 6 months?
📝 Response Preview: Based on enterprise cybersecurity reports from the last 6 months, here are three prominent threats:

1.  **Ransomware:** This continues to be a major threat, with attackers employing increasingly soph...

------------------------------

🎯 Optimized Query: Compare the environmental impact of electric vs hydrogen vehicles based on 2024 studies
📝 Response Preview: Comparing the environmental impact of electric vehicles (EVs) and hydrogen fuel cell vehicles (HFCVs) requires considering various factors, including greenhouse gas emissions, air quality, and resourc...

------------------------------

🎯 Optimized Query: What are the latest FDA approvals for cancer treatments announced this year?
📝 Response Preview: Here's a summary of some of the latest FDA approvals for cancer treatments announced in 2025:

**General Overview:**

*   From April 1 to June 30,

- Optimized query crafting: Demonstrates best practices like specifying exact time frames (“last 6 months”) and clear targets (“top 3 threats”).


### Understanding search behavior
To better understand how Gemini + Google Search behaves at runtime, it’s useful to collect basic analytics on queries and responses—especially in environments with variable user input like chatbots or customer support assistants.


In [12]:
# Example with detailed analysis
query = "What are the current semiconductor industry challenges and supply chain issues?"
print(f"🔍 Analyzing query: {query}")

# Execute the query with search enabled
response = llm.invoke(query, tools=[search_tool])

print("📊 Analysis:")
print(f"- Query length: {len(query)} characters")
print(f"- Response length: {len(response.content)} characters")
print(f"- Response type: {type(response)}")
print(f"- Contains recent information: {'2024' in response.content or '2025' in response.content}")

print("\n📝 Full Response:")
print(response.content)

🔍 Analyzing query: What are the current semiconductor industry challenges and supply chain issues?
📊 Analysis:
- Query length: 79 characters
- Response length: 3490 characters
- Response type: <class 'langchain_core.messages.ai.AIMessage'>
- Contains recent information: True

📝 Full Response:
Based on the search results, here are some of the current challenges and supply chain issues in the semiconductor industry as of July 2025:

**Challenges:**

*   **Talent Acquisition and Workforce Development:** A significant shortage of skilled workers, including engineers and technicians, is a major concern. This is due to the specialized and technical nature of chip design and manufacturing, as well as an aging workforce. Companies are investing in training, upskilling, and partnerships with educational institutions to address this gap.
*   **Geopolitical Tensions:** Rising tensions, particularly between the United States and China, and conflicts such as those in Ukraine/Russia and the Middle E

- Query inspection: Evaluates how long the query is, which can affect both processing time and model interpretation accuracy.
- Date keyword detection: Simple heuristics like checking for “2024” or “2025” help confirm that the search integration is pulling recent information rather than outdated content.